In [5]:
pickle_rel_path = '../datasets/notmnist/notMNIST.pickle'
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

### Import data

In [6]:
# Discover pickle path
file_dir = os.path.dirname(os.path.realpath('__file__'))
pickle_path = os.path.abspath(os.path.join(file_dir, pickle_rel_path))
print('Pickle location:',pickle_path)

# Load pickle
with open(pickle_path,'rb') as file:
    datasets = pickle.load(file)
print(datasets.keys())

# Set variables
train_dataset = datasets['train_dataset']
orig_train_labels = datasets['train_labels'] 
validate_dataset = datasets['valid_dataset']
orig_validate_labels = datasets['valid_labels'] 
test_dataset = datasets['test_dataset']
orig_test_labels = datasets['test_labels'] 

# Convert labels representation
num_labels = 10
def setup_labels(label_array, num_labels):
    return ((np.arange(num_labels) == label_array[:,None]).astype(np.float32))
train_labels = setup_labels(orig_train_labels, num_labels)
validate_labels = setup_labels(orig_validate_labels, num_labels)
test_labels = setup_labels(orig_test_labels, num_labels)

Pickle location: C:\Users\lgess\Documents\repo\learning_ai\datasets\notmnist\notMNIST.pickle
dict_keys(['test_dataset', 'train_labels', 'test_labels', 'train_dataset', 'valid_labels', 'valid_dataset'])


### Normalize data

Pre-normalized

### Define model

In [7]:
class Layer:
    
    def __init__(self, w_shape, b_shape, name=""):
        self.name = name    
        self.w_shape = w_shape
        self.b_shape = b_shape
        
    def create_vars(self):
        with tf.name_scope(self.name):
            initializer = tf.glorot_normal_initializer()
            self.w = tf.get_variable(name="W"+self.name, shape=self.w_shape, dtype=tf.float32, initializer=initializer)
            self.b = tf.get_variable(name="B"+self.name, shape=self.b_shape, dtype=tf.float32, initializer=initializer)  
    
    def weighted_inputs(self, x): 
        with tf.name_scope(self.name):
            return tf.matmul(x,tf.transpose(self.w))+self.b
    
    def activation(self, x):
        raise NotImplementedError()

class HiddenLayer(Layer):       
        
    def activation(self, x):
        with tf.name_scope(self.name):
             return tf.nn.relu(self.weighted_inputs(x))
                          
class OutputLayer(Layer):
        
    def activation(self, x):
        with tf.name_scope(self.name):
            return tf.nn.softmax(self.weighted_inputs(x))
                             
    def loss(self, x, y):
        with tf.name_scope(self.name):
            logits = self.weighted_inputs(x)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
            return tf.reduce_mean(cross_entropy)
        
class Model:
                          
    def __init__(self, layers, name=""):
        self.name = name
        self.layers = layers   
        
    def create_vars(self):
        for layer in self.layers:
            layer.create_vars()
                          
    def predict(self, x):
        for layer in self.layers:
            x = layer.activation(x)
        return x
                          
    def loss(self, x, y):  
        new_x = x
        for layer in self.layers[:-1]:
            new_x = layer.activation(new_x)
        return self.layers[-1].loss(new_x,y)
    
def create_simple_classifier(features_shape, num_neurons_array, num_labels):
                            
    layers = []    
    w_shape = None
    b_shape = features_shape
    index = 1

    for num_neurons in num_neurons_array:
        w_shape = np.append([num_neurons],b_shape)
        b_shape = [num_neurons]
        layers.append(HiddenLayer(w_shape, b_shape, name="Layer"+str(index)))
        index+=1

    w_shape = np.append([num_labels],b_shape)
    b_shape = [num_labels]
    layers.append(OutputLayer(w_shape, b_shape, name="Layer"+str(index))) 

    return Model(layers=layers)

In [41]:
class SessionRunner:
    
    def __init__(self, features_shape, num_neurons_array, num_labels):
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.session = tf.Session(config=tf.ConfigProto(log_device_placement=True))
            self.x = tf.placeholder(dtype=tf.float32,shape=np.append(None,features_shape), name="x")
            self.y = tf.placeholder(dtype=tf.float32,shape=np.append(None,[num_labels]), name="y")
            model = create_simple_classifier(features_shape, num_neurons_array, num_labels)
            model.create_vars()
            self.predict = model.predict(self.x)
            self.loss = model.loss(self.x,self.y)

    def train(self, train_dataset, train_labels, batch_size=200, learning_rate=0.5, iterations=1000):
        
        with self.graph.as_default():
            init = tf.global_variables_initializer()
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            step = optimizer.minimize(self.loss, name="Optimizer")  
        
        self.session.run(init)
        for i in range(iterations):        
            index = (i*batch_size)%(len(train_dataset)-batch_size)
            feed_dict={self.x: train_dataset[index:index+batch_size], 
                       self.y: train_labels[index:index+batch_size]}      
            self.session.run(step, feed_dict)
            if i%(iterations/10) == 0:
                loss_value = self.session.run(self.loss, feed_dict)
                print(loss_value)
        
    def evaluate(self, dataset, labels, num_predictions):
        predicted = self.session.run(self.predict, feed_dict={self.x: dataset[0:num_predictions]})
        expected = labels[0:num_predictions]
        print(self.__accuracy(predicted, expected))
        
    def __accuracy(self, predicted, expected):
        return np.mean(np.argmax(predicted,1) == np.argmax(expected,1))
        

### Train model

In [101]:
try:
    session_runner.session.close()
except:
    None
session_runner = SessionRunner(features_shape=[28*28], 
                               num_neurons_array = [200,100,50,25],
                               num_labels = (10))
session_runner.train(train_dataset.reshape([-1,28*28]), train_labels, batch_size=500, learning_rate=0.3, iterations=10000)
writer = tf.summary.FileWriter("/tf.log", session_runner.graph)

2.3206463
0.30876434
0.2633564
0.15529528
0.18212979
0.1489193
0.12042923
0.09344093
0.11788782
0.08389052


### Evaluate Model

In [102]:
session_runner.evaluate(train_dataset.reshape([-1,28*28]), train_labels, num_predictions=100000)
session_runner.evaluate(validate_dataset.reshape([-1,28*28]), validate_labels, num_predictions=100000)
#session_runner.evaluate(test_dataset.reshape([-1,28*28]), test_labels, num_predictions=test_dataset.shape[0])

0.94154
0.8899


In [103]:
session_runner.session.close()
writer.close()

### Test sample

In [ ]:
def show_sample(sample_image, label):
    plt.figure()
    plt.imshow(sample_image)
    print('Label:', label)

index=0
show_sample(validate_dataset[index],validate_labels[index])
sess.run(predict(),feed_dict={samples: [validate_dataset[index]]})